In [1]:
import pandas as pd

# Merging the same infobox
excel_path = pd.ExcelFile('D:\Kuliah\Skripsi\Ontologi-Sejarah-Indonesia\MichaelDawBalma\Code\infoboxes\infobox-orla.xlsx')
sheet_name = pd.read_excel(excel_path, 'military conflict1')
sheet_name2 = pd.read_excel(excel_path, 'Military Conflict2')
sheet_name3 = pd.read_excel(excel_path, 'military conflict')
csv_file_path = 'military-conflict-final.csv'

merged_df = pd.concat([sheet_name, sheet_name2, sheet_name3])

merged_df.to_csv(csv_file_path, index=False)

In [2]:
# csv.field_size_limit(100000000)

131072

In [3]:
import csv
military_conflict_clean = []

with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        military_conflict_clean.append(dict(row))

In [36]:
import geopy

def get_coordinates(place):
    geolocator = geopy.Nominatim(user_agent='indonesian-history-ontology')
    lct = geolocator.geocode(place, geometry='wkt')
    if lct:
        return lct.latitude, lct.longitude, lct.raw['geotext']
    else:
        return -0.789275, 113.921327, ''


for index, value in enumerate(military_conflict_clean):
    latitude, longitude, location = get_coordinates(value['place'])

    # Add latitude and longitude into the dictionary  
    value['latitude'] = latitude
    value['longitude'] = longitude
    value['location'] = location

In [65]:
'''
Merge column date_start-year_start and date_end-year_end into merged_date_start and merged_date_end
https://raw.githubusercontent.com/w3c/sdw/gh-pages/time/rdf/time.ttl (time:inXSDDate)
'''

month_list = {
    'Januari' : '01',
    'Februari' : '02',
    'Maret' : '03',
    'April' : '04',
    'Mei' : '05',
    'Juni' : '06',
    'Juli' : '07',
    'Agustus' : '08',
    'September' : '09',
    'Oktober' : '10',
    'November' : '11',
    'Desember' : '12',
}

defaultDateStart = '01-01'
defaultDateEnd = '12-31'

def check_date_format(date):
    # returns to dd format
    if len(date) == 1:
        return '0' + f'{date}'
    return date

for index, value in enumerate(military_conflict_clean):
    
    day = ''
    month = ''
    year = ''
    dateStartUsed = ''
    dateEndUsed = ''
    dt = ''
    dtSt = ''
    dtEnd = ''

    split_date = value['date'].split("-")
    
    # if it does not have end date
    if len(split_date) == 1 :
        dt = split_date[0].strip().split(" ")
        # if the dt is only the year
        if len(dt) == 1 :
            dateStartUsed = f"{dt[0]}-{defaultDateStart}"
            dateEndUsed = f"{dt[0]}-{defaultDateEnd}"
        else:
            day = dt[0]
            month = month_list[dt[1]]
            year = dt[2]
            
            day = check_date_format(day)
            dateStartUsed = f'{year}-{month}-{day}'
            dateEndUsed = dateStartUsed
            
    # if it does have end date
    else :
        dtSt = split_date[0].strip().split(" ")
        dtEnd = split_date[1].strip().split(" ")
        
        # end date were in good format
        day = dtEnd[0]
        month = month_list[dtEnd[1]]
        year = dtEnd[2]
        # change into dd format
        day = check_date_format(day)
        dateEndUsed = f"{year}-{month}-{day}"
        
        # start date and it is year only
        if len(dtSt) == 1:
            dateStartUsed = f"{dtSt[0]}-{defaultDateStart}"
        # start date without year; get end-date's year
        elif len(dtSt) == 2:
            day = check_date_format(dtSt[0])
            month = month_list[dtSt[1]]
            dateStartUsed = f"{dtEnd[2]}-{month}-{day}"
        else :
            day = check_date_format(dtSt[0])
            month = month_list[dtSt[1]]
            dateStartUsed = f"{dtSt[2]}-{month}-{day}"
    
    value['start-date'] = dateStartUsed
    value['end-date'] = dateEndUsed


In [72]:
import wikipedia

def retrieve_paragraph(article_title):
    wikipedia.set_lang("id")
    return wikipedia.summary(article_title, auto_suggest=False)

def get_url_wikipedia(article_title):
    wikipedia.set_lang("id")
    page = wikipedia.page(article_title, auto_suggest=False)
    return page.url


In [75]:
for index, data in enumerate(military_conflict_clean[:10]):
    # if the title has more than paragraph summary, get the first summary
    summary = retrieve_paragraph(data['page_title']).split("\n")
    data['summary'] = summary[0]
    
    wikiurl = get_url_wikipedia(data['page_title'])
    data['wikiurl'] = wikiurl

In [76]:
for index, data in enumerate(military_conflict_clean[10:]):
    # if the title has more than paragraph summary, get the first summary
    summary = retrieve_paragraph(data['page_title']).split("\n")
    data['summary'] = summary[0]

    wikiurl = get_url_wikipedia(data['page_title'])
    data['wikiurl'] = wikiurl

In [84]:
fieldnames = military_conflict_clean[0].keys()

with open('military-conflict-final-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    writer.writerows(military_conflict_clean)

### Handling Actor on Multivalue Commander 

In [52]:
# Handling page_title, wikiurl, and summary (if any) for commanders from a 'Military Conflict' Event. (For Person mapping in timeline)
# Birth-Death Date and image will be handled manually.

import csv
commander1 = []
commander2 = []

with open('commander1.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        commander1.append(dict(row))

with open('commander2.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        commander2.append(dict(row))



In [53]:
import wikipedia

for value in commander1:
    if value['commander1'] != '' :
        wikipedia.set_lang("id")
        try:
            page = wikipedia.page(value['commander1'], auto_suggest=False)
            value['page_title'] = page.title
            value['summary'] = page.summary.split("\n")[0]
            value['wikiurl'] = page.url
        except wikipedia.PageError :
            value['page_title'] = ''
            value['summary'] = ''
            value['wikiurl'] = ''

for value in commander2:
    if value['commander2'] != '' :
        wikipedia.set_lang("id")
        try:
            page = wikipedia.page(value['commander2'], auto_suggest=False)
            value['page_title'] = page.title
            value['summary'] = page.summary.split("\n")[0]
            value['wikiurl'] = page.url
        except wikipedia.PageError :
            value['page_title'] = ''
            value['summary'] = ''
            value['wikiurl'] = ''

In [54]:
fieldnames = commander1[0].keys()
with open('commander1-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(commander1)

In [55]:
fieldnames2 = commander2[0].keys()
with open('commander2-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames2)
    writer.writeheader()
    writer.writerows(commander2)

### Handle Vocab Changes and Merging Vocab

In [14]:
import csv
csv.field_size_limit(100000000)
military_conflict_clean = []

with open('military-conflict-final-cleaned.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        military_conflict_clean.append(dict(row))

In [15]:
for military_conflict in military_conflict_clean :
    casualties = []
    if military_conflict['casualties1']:
        casualties.append(military_conflict['casualties1'])
    if military_conflict['casualties2']:
        casualties.append(military_conflict['casualties2'])
    if military_conflict['casualties3']:
        casualties.append(military_conflict['casualties3'])
    military_conflict['casualties'] = ', '.join(casualties)
    
    del military_conflict['casualties1']
    del military_conflict['casualties2']
    del military_conflict['casualties3']


In [16]:
fieldnames = military_conflict_clean[0].keys()

with open('military-conflict-final-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    writer.writerows(military_conflict_clean)

In [17]:
# Combatant
combatant1 = []
combatant2 = []
with open('combatant1.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        combatant1.append(dict(row))
with open('combatant2.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        combatant2.append(dict(row))

In [30]:
merged_list = []

for f_entry, s_entry in zip(combatant1, combatant2):
    key = f_entry['key']
    side = ', '.join(filter(None, [f_entry['combatant1'], s_entry['combatant2']]))
    merged_list.append({'key': key, 'side': side})

In [32]:
fieldnames = ['key', 'side']

with open('combatant-final.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for entry in merged_list:
        writer.writerow(entry)